In [49]:
setwd("/Users/rebecca/sudmant/analyses/myotis/analysis/reciprocal_best_hits/01_get_best_hits")

library(pafr)
library(dplyr, quietly = TRUE)
library(stringr)
library(data.table)

options(scipen = 100, digits = 10)
myo_meta <- read.csv("../../../data/myotis_meta.csv")

In [ ]:
for (i in seq_along(myo_meta$Abbr)) {
  
  for (j in seq_along(myo_meta$Abbr)) {

    if (i < j) {
      
      print(paste(myo_meta$Abbr[i], "vs.", myo_meta$Abbr[j]))

      # working_paf1 <- read_paf(file.path("resources/pafs/trimmed", intersect(
      #   list.files(path = "resources/pafs/trimmed", pattern = paste0("^", myo_meta$Abbr[j])),
      #   list.files(path = "resources/pafs/trimmed", pattern = paste0("_", myo_meta$Abbr[i], ".*_trimmed.paf"))
      # )), include_tags = FALSE)

      # working_paf2 <- read_paf(file.path("resources/pafs/trimmed", intersect(
      #   list.files(path = "resources/pafs/trimmed", pattern = paste0("^", myo_meta$Abbr[i])),
      #   list.files(path = "resources/pafs/trimmed", pattern = paste0("_", myo_meta$Abbr[j], ".*_trimmed.paf"))
      # )), include_tags = FALSE)

      working_paf1 <- read_paf(file.path("resources/pafs", intersect(
        list.files(path = "resources/pafs", pattern = paste0("^", myo_meta$Abbr[j])),
        list.files(path = "resources/pafs", pattern = paste0("_", myo_meta$Abbr[i]))
      )), include_tags = FALSE)

      working_paf2 <- read_paf(file.path("resources/pafs", intersect(
        list.files(path = "resources/pafs", pattern = paste0("^", myo_meta$Abbr[i])),
        list.files(path = "resources/pafs", pattern = paste0("_", myo_meta$Abbr[j]))
      )), include_tags = FALSE)

      spec1_intersect <- fread(file.path("resources/beds", paste0(
        myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], "_positions_intersect.bed"
      )), data.table = FALSE, stringsAsFactors = FALSE)

      spec2_intersect <- fread(file.path("resources/beds", paste0(
        myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], "_positions_intersect.bed"
      )), data.table = FALSE, stringsAsFactors = FALSE)

      working_paf1$qname <- gsub("SCAF", "SUPER", working_paf1$qname)
      working_paf2$qname <- gsub("SCAF", "SUPER", working_paf2$qname)
      working_paf1$tname <- gsub("SCAF", "SUPER", working_paf1$tname)
      working_paf2$tname <- gsub("SCAF", "SUPER", working_paf2$tname)
      spec1_intersect$V1 <- gsub("SCAF", "SUPER", spec1_intersect$V1)
      spec1_intersect$V8 <- gsub("SCAF", "SUPER", spec1_intersect$V8)
      spec2_intersect$V1 <- gsub("SCAF", "SUPER", spec2_intersect$V1)
      spec2_intersect$V8 <- gsub("SCAF", "SUPER", spec2_intersect$V8)

      working_paf1$Id <- with(working_paf1, paste(qstart, qend, tstart, tend, sep = "_"))      
      working_paf2$Id <- with(working_paf2, paste(qstart, qend, tstart, tend, sep = "_"))   
      working_paf1$Rev_Id <- with(working_paf1, paste(tstart, tend, qstart, qend, sep = "_"))      
      working_paf2$Rev_Id <- with(working_paf2, paste(tstart, tend, qstart, qend, sep = "_"))   

      ## Insert new sequences into original PAF files:

      target_inds <- match(working_paf1$Id, spec1_intersect$V7)
      working_paf1$tstart <- spec1_intersect$V2[target_inds]
      working_paf1$tend <- spec1_intersect$V3[target_inds]
      query_inds <- match(working_paf1$Id, spec2_intersect$V15)
      working_paf1$qstart <- spec2_intersect$V2[query_inds]
      working_paf1$qend <- spec2_intersect$V3[query_inds]

      target_inds <- match(working_paf2$Id, spec2_intersect$V7)
      working_paf2$tstart <- spec2_intersect$V2[target_inds]
      working_paf2$tend <- spec2_intersect$V3[target_inds]
      query_inds <- match(working_paf2$Id, spec1_intersect$V15)
      working_paf2$qstart <- spec1_intersect$V2[query_inds]
      working_paf2$qend <- spec1_intersect$V3[query_inds]

      working_paf1 <- working_paf1[!is.na(working_paf1$tstart),]
      working_paf2 <- working_paf2[!is.na(working_paf2$tstart),]

      ## Subset to hits found in both directions:

      working_paf1 <- working_paf1[with(working_paf1, paste(qstart, qend, tstart, tend)) %in% with(working_paf2, paste(tstart, tend, qstart, qend)),]
      working_paf2 <- working_paf2[with(working_paf2, paste(qstart, qend, tstart, tend)) %in% with(working_paf1, paste(tstart, tend, qstart, qend)),]
      working_paf2 <- working_paf2[match(with(working_paf1, paste(tstart, tend, qstart, qend)), with(working_paf2, paste(qstart, qend, tstart, tend))),]
      working_paf1 <- working_paf1[match(with(working_paf2, paste(tstart, tend, qstart, qend)), with(working_paf1, paste(qstart, qend, tstart, tend))),]

      ## Save results:
      
      working_paf1$qname <- as.character(working_paf1$qname)
      working_paf2$qname <- as.character(working_paf2$qname)
      working_paf1$qname <- gsub("#", ".", working_paf1$qname, fixed = TRUE)
      working_paf2$qname <- gsub("#", ".", working_paf2$qname, fixed = TRUE)

      write.csv(working_paf1, file = paste0(
        "results/", myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], "_best_hits.csv"
      ), row.names = FALSE, quote = FALSE)

      write.csv(working_paf2, file = paste0(
        "results/", myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], "_best_hits.csv"
      ), row.names = FALSE, quote = FALSE)

    }
  }
}